In [23]:
import mysql.connector
from dotenv import load_dotenv
import os
import json
import hashlib


def leeactzl(user, repo_name, path_to_file):
    json_url ='https://api.github.com/repos/{}/{}/contents/{}'.format(user, repo_name,
                                                                      path_to_file)
    response = requests.get(json_url) #get data from json file located at specified URL 

    if response.status_code == requests.codes.ok:
        jsonResponse = response.json()  # the response is a JSON
        #the JSON is encoded in base 64, hence decode it
        content = base64.b64decode(jsonResponse['content'])
        #convert the byte stream to string
        jsonString = content.decode('utf-8')
        return json.loads(jsonString)
 
    else:
        return 'Content was not found.'


def conexion():
    load_dotenv()

    sql_id = os.getenv("SQL_ID")
    sql_pw = os.getenv("SQL_PW")
    sql_hs = os.getenv("SQL_HOST")
    sql_db = os.getenv("SQL_DB")
    cnx = mysql.connector.connect(user=sql_id, password=sql_pw, host=sql_hs, database=sql_db)
    return cnx


user = 'sistelca'
repo_name = 'desechosSolidos'
file_orig = "cibercom_orig_data_actzl.json"
file_orig_check = "cibercom_orig_data_actzl.cheqsum.txt"
file_dest_check = "cibercom_dest_data_actzl.cheqsum.txt"

cheq_orig  = leeactzl(user, repo_name, file_orig_check)
cheq_dest = leeactzl(user, repo_name, file_dest_check)


if cheq_orig == cheq_dest:

    cnx = conexion()
    cursor = cnx.cursor()
    query = ("select * from Actzl where pasar=2")
    cursor.execute(query)
    result = cursor.fetchall()

    # sequence = cursor.column_names
    # print(sequence)

    datos = []
    ides = []
    for x in result:
        datos.append({'fecha': str(x[1]), 'instruccion': x[2], 'firma': x[3]})
        ides.append(x[0])

    orig = json.dumps(datos)
    check_orig = hashlib.sha1(orig.encode('utf-8')).hexdigest()

    with open(file_orig, 'w', encoding = 'utf-8') as f:
        json.dump(datos, f)

    with open(file_orig_check, 'w', encoding = 'utf-8') as f:
        f.write(check_orig)

    for x in ides:
        query = ("UPDATE Actzl SET pasar=0 WHERE id = {}").format(x)
        cursor.execute(query)

    cnx.commit()
    cursor.close()
    cnx.close()